In [11]:
import numpy as np
import pandas as pd

# Day 1

In [66]:
data = np.array(np.loadtxt('day1_input.txt', dtype='int'))

#part 1
inc = 0
for i in np.arange(len(data)-1): #len(data)
    if(data[i+1] > data[i]): inc += 1
print("Single Increase: ", inc)

#part 2
inc = 0
for i in np.arange(len(data)-3):
    first = sum([data[i], data[i+1], data[i+2]])
    second = sum([data[i+1], data[i+2], data[i+3]])
#     print(data[i], first, second)
    if(second > first):inc +=1
print("Window Increase: ",inc)

Single Increase:  1451
Window Increase:  1395


# Day 2